# **ROB'S NYTIMES COVID ANALYSIS (PER COUNTY)**


- import the NYTIMES covid  data as a pandas frame
- massage and plot
- **To Execute**:
 - select Runtime->Run all
 - or Type Shift-Return on each cell to execute it



In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.subplots as subplots
# I'd prefer matplotlib notebook for the backend, but it doesnt work under colab
#%matplotlib inline

# reload modules without reloading explicitly
import importlib
%load_ext autoreload
%autoreload 2

import rycovid as cvd

In [2]:
%pwd


'/Users/ryu/Documents/p2020/covid'

## LOAD DATA

see https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92



Load the NYTIMES data from the git repo raw view

In [3]:
#
# set the URL for the "raw view" of the data
NYTIMES_COUNTY_URL="https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"


In [4]:
# get data direct
nyt_df = pd.read_csv(NYTIMES_COUNTY_URL)

# and write it again locally
tmpfile = open('us-counties-localcopy.csv',mode='w',newline='')
nyt_df.to_csv(tmpfile)
tmpfile.close()

read county population data from the US Census excel spreadsheet

In [5]:
!ls
pop_df = pd.read_excel('co-est2019-annres.xlsx',header=3,nrows=3147-4)
pop_df = pop_df.rename(columns={"Unnamed: 0":"county+state"})
pop_df = pop_df.drop([0]).reset_index()  # first row is overall US population
pop_df.head()
print(len(pop_df))


README.html                        co-est2019-annres.xlsx
README.md                          nyt-us-counties.localcopy.csv
README.md~                         rycovid.py
__pycache__                        rycovid.py~
analyze-nytimes-covid-county.ipynb state-geocodes-v2017.xlsx
applemobilitytrends-2020-04-12.csv us-counties-localcopy.csv
cases-vs-date.png
3142


## NORMALIZE DATA SETS

In [6]:
# 
# in census data, split "county,state" columns into separate columns
pop_df1=pop_df.copy()
pop_df1 = cvd.split_county_state(pop_df1)
assert 'county' in pop_df1
assert 'state' in pop_df1

3142
3142


In [7]:
# 
# census data: bunch of reformatting and normalization
pop_df2 = pop_df1.copy()
pop_df2=cvd.strip_leading_dot(pop_df2,'county')
pop_df2 = cvd.to_lower(pop_df2,"county")
pop_df2 = cvd.to_lower(pop_df2,"state")
#pop_df2 = strip_trailing_county(pop_df2,'county')
pop_df2 = cvd.strip_spaces(pop_df2,'county')
pop_df2 = cvd.strip_spaces(pop_df2,'state')

pop_df2 = cvd.do_misc_census_fixup(pop_df2)


index=cvd.get_index_county_state(pop_df2, 'new york city', 'new york')
assert len(pop_df2[index]['county'])==1

In [8]:
#
#  NYT data: reformatting and normalization
nyt_df1 = nyt_df.copy()
nyt_df1 = cvd.to_lower(nyt_df1,'county')
nyt_df1 = cvd.to_lower(nyt_df1,'state')

#
# drop all county='unknown' entries
unknown_indexes = nyt_df1[nyt_df1.county=='unknown'].index    
nyt_df1 = nyt_df1.drop(index=unknown_indexes)
nyt_df1[nyt_df1.county=='unknown']

# convert 'date' to timestamp and daynum
nyt_df1 = cvd.fix_date(nyt_df1)

nyt_df1 = cvd.do_misc_nyt_fixup(nyt_df1)

assert len(nyt_df1[nyt_df1.county=='new york'])==0

In [9]:
#
#verify that every county, state in NYT exists in census data
#
mismatch_list = cvd.validate_county_match(nyt_df1,pop_df2)
assert len(mismatch_list)==0, f"found {len(mismatch_list)} (county,state) mismatches"


# **MANIPULATE DATA**

In [10]:
#
# NYT: drop data earlier than START_DATE
START_DATE=pd.to_datetime('2020-03-10')  # find index corresponding to START_DATE

nyt_df2 = nyt_df1[nyt_df1.tstamp >= START_DATE]
nyt_df2 = nyt_df2.reset_index(drop=True)

assert nyt_df2.loc[0,'tstamp']==START_DATE, "first entry does not correspond to desired date"

In [11]:
#
# NYT: determine STOP_DATE
last_entry = nyt_df2.iloc[-1]
STOP_DATE=last_entry['tstamp']
assert STOP_DATE > START_DATE

In [12]:
#
# generate list of ALL (county, state) tuples in NYT data
just_sc_df = nyt_df2.loc[:,['county','state']]
just_sc_df = just_sc_df.drop_duplicates()
all_cs_list = [(row['county'],row['state']) for index, row in just_sc_df.iterrows()]

print (f"NYT data has {len(all_cs_list)} county,state tuples")
assert len(all_cs_list) >= 2486

NYT data has 2993 county,state tuples


In [13]:
#
# generate list of all (county, state) tuples for timestamp==STOP_DATE
latest_df = nyt_df2[nyt_df2.tstamp==STOP_DATE]
latest_cs_list = [(row['county'],row['state']) for index, row in latest_df.iterrows()]
assert len(latest_df[latest_df.county=='unknown'])==0
print (f"NYT data @{STOP_DATE} has {len(latest_cs_list)} county,state tuples")
assert len(latest_cs_list) <= len(all_cs_list)


NYT data @2020-06-09 00:00:00 has 2990 county,state tuples


In [14]:
# out of curiousity, what county+state dropped out of the latest county+state list?
for county, state in all_cs_list:
    x = (county, state)
    if x not in latest_cs_list:
        print(f"latest_cs_list missing {x}")
    #end
#end
#nyt_df2[(nyt_df2.county=='richmond') & (nyt_df2.state=='north carolina')].tail()

latest_cs_list missing ('harlan', 'nebraska')
latest_cs_list missing ('modoc', 'california')
latest_cs_list missing ('throckmorton', 'texas')


In [15]:
#
# FOR THE LATEST NYT ENTRIES, compute norm_deaths_per_county_df and norm_cases_per_county_df
latest_df = nyt_df2[nyt_df2.tstamp==STOP_DATE]
print(len(latest_df))
#print(latest_df.iloc[-1])
normed_df = cvd.normalize_cases_deaths(latest_df, pop_df2) 

assert len(latest_df)==len(normed_df)
assert 'norm_cases' in normed_df
assert 'norm_deaths' in normed_df

2990


In [16]:
# Uncomment the county selection criteria below

#COUNTY_CRITERIA="most_cases_per_cap"  # "cases"
#COUNTY_CRITERIA="most_deaths_per_cap" 
COUNTY_CRITERIA="most_cases" 
#COUNTY_CRITERIA="most_deaths" 
#COUNTY_CRITERIA="manual_selection"


if COUNTY_CRITERIA=="most_deaths_per_cap":
    most_deaths_df = normed_df.sort_values('norm_deaths', ascending=False).head(10)
    cs_list = [(row['county'],row['state']) for index, row in most_deaths_df.iterrows()]
elif COUNTY_CRITERIA=='most_cases_per_cap':
    most_cases_df = normed_df.sort_values('norm_cases', ascending=False).head(10)
    cs_list = [(row['county'],row['state']) for index, row in most_cases_df.iterrows()]
elif COUNTY_CRITERIA=='most_cases':
    most_cases_df = nyt_df2[nyt_df2.tstamp==STOP_DATE].sort_values('cases',ascending=False).head(10)
    cs_list = [(row['county'],row['state']) for index, row in most_cases_df.iterrows()]
elif COUNTY_CRITERIA=='most_deaths':
    most_cases_df = nyt_df2[nyt_df2.tstamp==STOP_DATE].sort_values('deaths',ascending=False).head(10)
    cs_list = [(row['county'],row['state']) for index, row in most_cases_df.iterrows()]
elif COUNTY_CRITERIA=='manual_selection':    
    cs_list = [('new york city', 'new york'),('alachua','florida')ho]
else:
    assert False, "invalid selection"
#end
cs_list.append(('santa clara','california'))
#cs_list.append(('minnehaha', 'south dakota'))
print(f"selected {len(cs_list)} counties")
assert len(cs_list) > 0

selected 11 counties


In [17]:
# verify that entries in cs_list exist in NYT data
for cs in cs_list:
    m = cvd.get_index_county_state(pop_df2,cs[0],cs[1],smartmatch=False) # do exact match
    t = len(pop_df2[m])
    assert t>0, f"could not find tuple ({cs[0]},{cs[1]}) in census data"

In [18]:
#
# discard (county,state) rows not in our list
nyt_df3 = cvd.filter_rows_by_state_county(nyt_df2, cs_list).reset_index()
print(len(nyt_df3))

assert len(nyt_df3[nyt_df3.county==cs_list[0][0]]) > 0, f"did not find rows with county={cs_list[0][0]}"


1011


In [39]:
# calculate delta quantities
# compute deltacases (i.e. Nd)
nyt_df4 = nyt_df3.copy()    
nyt_df4 = cvd.normalize_cases_deaths(nyt_df4, pop_df2)

for (county,state) in cs_list:
    nyt_df4 = cvd.calc_delta_over_daynum(nyt_df4, county, state, 'cases', 'dcases')
    nyt_df4 = cvd.calc_delta_over_daynum(nyt_df4, county, state, 'deaths', 'ddeaths')   
    nyt_df4 = cvd.calc_growthfactor(nyt_df4, county, state)
#end
assert 'norm_cases' in nyt_df4
assert 'norm_deaths' in nyt_df4
assert 'ddeaths' in nyt_df4
assert 'dcases' in nyt_df4
assert 'growthfactor' in nyt_df4





created column norm_deaths
created column norm_cases


In [40]:
nyt_df4.head()

,index,county,state,fips,cases,deaths,tstamp,daynum,norm_cases,norm_deaths,dcases,ddeaths,growthfactor
0,103,new york city,new york,NaN,37,0,2020-03-10,70,0.000023,0.000000e+00,37.0,0.0,0.000000
1,289,new york city,new york,NaN,52,0,2020-03-11,71,0.000032,0.000000e+00,15.0,0.0,0.405405
2,523,new york city,new york,NaN,96,0,2020-03-12,72,0.000059,0.000000e+00,44.0,0.0,2.933333
3,814,new york city,new york,NaN,155,0,2020-03-13,73,0.000095,0.000000e+00,59.0,0.0,1.340909
4,1160,new york city,new york,NaN,269,1,2020-03-14,74,0.000165,6.139844e-07,114.0,1.0,1.932203


## FINALLY, PLOTS

In [41]:
nytplot_df = nyt_df4.copy()

In [42]:
# graph: total cases per day
GRAPH_TITLE=f"cases vs date - {COUNTY_CRITERIA}"
graph_list=[]
for county,state in cs_list:
    indices=(nytplot_df.county==county) & (nytplot_df.state==state)
    tmp_df = nytplot_df[indices]
    graph_list.append(go.Scatter(name=f"{county}, {state}",
                               x=tmp_df['tstamp'],
                               y=tmp_df['cases'],
                               mode='lines+markers'
                               ))
x_axis=dict(
    showgrid=True,
    title='day')
y_axis=dict(
    showgrid=True,
    title='cases')

layout=go.Layout(
    title={
        'text':GRAPH_TITLE,
        'x':0.5  # center
    },
    xaxis=x_axis,
    yaxis_type="log",
    yaxis=y_axis,
    width=1000,
    showlegend=True
)
fig = go.Figure( data=graph_list, layout=layout)
fig.update_yaxes(gridcolor='black')
fig.update_xaxes(gridcolor='black')
fig.show()


In [45]:
# graph: total deaths per day
GRAPH_TITLE=f"deaths vs date - {COUNTY_CRITERIA}"
graph_list=[]
for county,state in cs_list:
    indices=(nytplot_df.county==county) & (nytplot_df.state==state)
    tmp_df = nytplot_df[indices]
    graph_list.append(go.Scatter(name=f"{county}, {state}",
                               x=tmp_df['tstamp'],
                               y=tmp_df['deaths'],
                               mode='lines+markers'
                               ))
x_axis=dict(
    showgrid=True,
    title='day')
y_axis=dict(
    showgrid=True,
    title='cases')

layout=go.Layout(
    title={
        'text':GRAPH_TITLE,
        'x':0.5  # center
    },
    xaxis=x_axis,
    yaxis_type="log",
    yaxis=y_axis,
    width=1000,
    showlegend=True
)
fig = go.Figure( data=graph_list, layout=layout)
fig.update_yaxes(gridcolor='black')
fig.update_xaxes(gridcolor='black')
fig.show()


In [48]:
#
# plot new cases vs total cases
# this plot supposedly highlights when a population drops off the curve
GRAPH_TITLE=f"cases-per-day vs total_cases - {COUNTY_CRITERIA}"
graph_list=[]
for county,state in cs_list:
    indices=(nytplot_df.county==county) & (nytplot_df.state==state)
    tmp_df = nytplot_df[indices]
    graph_list.append(go.Scatter(name=f"{county}, {state}",
                               x=tmp_df['cases'],
                               y=cvd.avg_over_daynum(tmp_df, county,state,'dcases',7),
                               mode='lines+markers'
                               ))
x_axis=dict(
    showgrid=True,
    title='total cases')
y_axis=dict(
    showgrid=True,
    title='new cases')
layout=go.Layout(
    title={
        'text':GRAPH_TITLE,
        'x':0.5  # center
    },
    xaxis=x_axis,
    xaxis_type="log",
    yaxis=y_axis,
    yaxis_type="log",
    width=1000,
    showlegend=True
)
fig = go.Figure( data=graph_list, layout=layout)
fig.update_yaxes(gridcolor='black')
fig.update_xaxes(gridcolor='black')
fig.show()

In [52]:
#
# new deaths per day
GRAPH_TITLE=f"new cases vs date - {COUNTY_CRITERIA}"
graph_list=[]
for county,state in cs_list:
    indices=(nytplot_df.county==county) & (nytplot_df.state==state)
    tmp_df = nytplot_df[indices]
    graph_list.append(go.Scatter(name=f"{county}, {state}",
                               x=tmp_df['tstamp'],
                               y=tmp_df['dcases'],
                               mode='lines+markers'
                               ))
x_axis=dict(
    showgrid=True,
    title='day')
y_axis=dict(
    showgrid=True,
    title='cases')

layout=go.Layout(
    title={
        'text':GRAPH_TITLE,
        'x':0.5  # center
    },
    xaxis=x_axis,
    #yaxis_type="log",
    yaxis=y_axis,
    width=1000,
    showlegend=True
)
fig = go.Figure( data=graph_list, layout=layout)
fig.update_yaxes(gridcolor='black')
fig.update_xaxes(gridcolor='black')
fig.show()

In [54]:
#
# growthfactor per day
GRAPH_TITLE=f"growthfactor vs date - {COUNTY_CRITERIA}"
graph_list=[]
for county,state in cs_list:
    indices=(nytplot_df.county==county) & (nytplot_df.state==state)
    tmp_df = nytplot_df[indices]
    graph_list.append(go.Scatter(name=f"{county}, {state}",
                               x=tmp_df['tstamp'],
                               y=cvd.avg_over_daynum(tmp_df, county, state, 'growthfactor',14),
                               mode='lines+markers'
                               ))
x_axis=dict(
    showgrid=True,
    title='day')
y_axis=dict(
    showgrid=True,
    range=[0,3],
    title='growthf')

layout=go.Layout(
    title={
        'text':GRAPH_TITLE,
        'x':0.5  # center
    },
    xaxis=x_axis,
    #yaxis_type="log",
    yaxis=y_axis,
    width=1000,
    showlegend=True
)
fig = go.Figure( data=graph_list, layout=layout)
fig.update_yaxes(gridcolor='black')
fig.update_xaxes(gridcolor='black')
fig.show()